<a href="https://colab.research.google.com/github/finiteautomata/rioplatense_hate_speech/blob/main/notebooks/Mixtral_Colab_(run_only_in_A100).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pysentimiento accelerate bitsandbytes datasets



In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
)


/home/jmperez/miniconda3/envs/rioplatense_hs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 19/19 [00:23<00:00,  1.21s/it]


In [61]:
from IPython.display import display, clear_output, HTML
import textwrap

prompt = "[INST] Explicar cuántos bytes de memoria de GPU necesito para correr un modelo de lenguaje de 7 billones de parámetros [/INST]"
inputs = tokenizer(prompt, return_tensors="pt")
next_token = None
new_tokens = []
while next_token != tokenizer.eos_token_id:
    output = model.generate(**inputs, max_new_tokens=1, pad_token_id=tokenizer.eos_token_id)

    next_token = output[0][-1].item()
    inputs = {
        "input_ids": output,
        "attention_mask": torch.ones_like(output),
    }
    new_tokens.append(next_token)
    text = tokenizer.decode(new_tokens, skip_special_tokens=True)

    #text = "\n".join(textwrap.wrap(text, width=100))
    clear_output(wait=True)
    display(HTML(text))


In [36]:
new_tokens

[1700, 8752, 4526, 340, 550, 28754, 2854, 325, 11761, 20575, 11559]

In [49]:
tokenizer.decode

False

In [2]:
prompt = """[INST] Determinar si el siguiente mensaje contiene discurso de odio.

Entendemos que hay discurso de odio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:

- mujer: mujeres o movimiento feminista
- lgbti: contra gays, lesbianas, transexuales y otras identidades de género
- racismo: inmigrantes, xenofobia, o contra pueblos aborígenes
- clase: personas de bajos recursos o motivos de clase
- discapacidad: discapacidad, problemas de salud mental o de adicción al alcohol u otras drogas
- política: ideología política
- aspecto: aspecto, gordofobia o edad
- criminal: presos o delincuentes comunes

Responder una o varias de las características separadas por coma, o bien "nada" si no hay discurso de odio. Pensar paso a paso la respuesta antes de responder.
[/INST]
contexto: Al borde del llanto, Nati Jota pidió disculpas por sus repudiables tuits pero los justificó: 'Quería sumar seguidores'
texto: Show o nada mi amorrr!! Sos una imbesil!
salida: Es un insulto y no menciona nada relacionado a las características protegidas. La respuesta final es 'nada'
---
contexto: Violento motín en la cárcel de Devoto: tomaron e incendiaron pabellones
texto: Hermanito lo que son esos grones. Ahí adentro le robaron la billetera al coronavirus
salida: La palabra "grones" significa negros, lo cual alude a su color de piel y aspecto. Además, se refiere a su condición de presos. La respuesta final es "criminal, racismo, aspecto"
---
contexto: Mayra Mendoza criticó el banderazo contra el Gobierno: "No tiene sentido y es antidemocrático"
texto: Que negrita villera....ni con la plata q roba puede arreglarse
salida: El texto alude a su color de piel, a su condición social (villera), y a su aspecto. La respuesta final es "racismo, aspecto, clase"
---
contexto: Les darán DNI provisorio a personas en situación vulnerable, para que puedan empezar a cobrar planes sociales
texto: Seguimos alimentando vagos
salida: El comentario se refiere a quienes cobran planes sociales como vagos. La respuesta final es "clase"
---
contexto: Otro ataque de furia contra un colectivero: una mujer trans le pateó la unidad y le tiró un ladrillazo
texto: Un tipo operado. Con la fuerza de un hombre y no la de una mujer
salida: El texto alude a que la mujer trans es un hombre. La respuesta final es "lgbti"
---
contexto: Elisa Carrió denunció que el Gobierno usa la pandemia para "establecer un estado de sitio"
texto: Gorda psiquiátrica
salida: El texto alude a su aspecto (gorda) y la acusa de tener problemas psiquiátricos. La respuesta final es "aspecto, discapacidad"
---
contexto: Los dos presos heridos de bala en el motín de Devoto tienen Covid-19 y uno quedó hemipléjico
texto: justicia divina!
salida: El texto alude a que los presos merecen ser baleados. La respuesta final es 'criminal'
</s>
[INST]
contexto: {contexto}
texto: {texto}
[/INST]
"""




In [3]:
from datasets import load_dataset

dataset_name = "piuba-bigdata/contextualized_hate_speech"

ds = load_dataset(dataset_name)

In [4]:
test_ds = ds["test"].shuffle()

In [5]:
from pysentimiento.preprocessing import preprocess_tweet
import re

url_regex = r"\burl\b"


def text_preprocess(text):
    text = preprocess_tweet(text, preprocess_hashtags=False, demoji=False)
    text = text.replace("@usuario", "")

    text = re.sub(url_regex, "", text)
    # Replace multiple spaces with one
    text = " ".join(text.split())
    text = text.replace("\n", " ")

    return text


In [6]:
tokenizer.model_max_length = 6400
tokenizer.pad_token_id = tokenizer.eos_token_id

In [7]:
def tokenize(example):
    texto = text_preprocess(example["text"])

    contexto = preprocess_tweet(
        example["context_tweet"],
        preprocess_hashtags=False,
        demoji=False,
        preprocess_handles=False,
    )
    model_input = prompt.format(contexto=contexto, texto=texto)
    return tokenizer(model_input, truncation=True)

tokenized_ds = test_ds.map(tokenize, batched=False)

Map: 100%|██████████| 11343/11343 [00:12<00:00, 904.90 examples/s] 


In [8]:
# prompt: Sort tokenized_ds by length of input_ids
tokenized_ds = tokenized_ds.map(lambda x: {"len": len(x["input_ids"])}, batched=False)

sorted_tokenized_ds = tokenized_ds.sort("len")

Map: 100%|██████████| 11343/11343 [00:05<00:00, 2027.51 examples/s]


In [9]:
# prompt: Build a dataloader with a Causal LLM collator

from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
batch_size = 16
collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

def collate(inputs):
    attention = [ex["attention_mask"] for ex in inputs]
    input_ids = [ex["input_ids"] for ex in inputs]
    ids = [ex["id"] for ex in inputs]
    return ids, tokenizer.pad({"input_ids": input_ids, "attention_mask": attention}, return_tensors="pt")


dataloader = DataLoader(
    sorted_tokenized_ds,
    batch_size=batch_size,
    collate_fn=collate,
    shuffle=False,
    pin_memory=True,
    num_workers=16,
)


In [10]:
from tqdm.auto import tqdm


for ids, inputs in tqdm(dataloader):
    output = model.generate(**inputs, max_new_tokens=150)

    for k, id in enumerate(ids):
        output_text = tokenizer.decode(output[k], skip_special_tokens=True)

        #with open(path+"/{}.txt".format(id), "w") as f:
        #    f.write(output_text)

    #for _ in range(4):
    #    print("="*80)
    #print(f"contexto: {contexto}")
    #print(f"texto: {texto}")
    #print(output_text)


  0%|          | 0/709 [00:00<?, ?it/s]/home/jmperez/miniconda3/envs/rioplatense_hs/lib/python3.10/site-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
  0%|          | 3/709 [04:43<18:32:31, 94.55s/it]


KeyboardInterrupt: 